In [2]:
# import needed modules

import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os
import pandas as pd

In [3]:
spark = (SparkSession.builder.appName("Pride and Prejudice word count").
           getOrCreate())

In [ ]:
result = (spark.read.text("./1342-0.txt").
select(F.split(F.col("value"), " ").alias("split_word")).
select(F.explode(F.col("split_word")).alias("explode_word")).
select(F.lower(F.col("explode_word")).alias("lower_word")).
select(F.regexp_extract(F.col("lower_word"), "[a-z]*", 0).alias("extract_word")).
where((F.col("extract_word") != "")&(~F.col("extract_word").isin('the', 'in'))&(F.length(F.col("extract_word")) > 3)).
groupby(F.col("extract_word")).
count()
)
#result.coalesce(1).write.csv("C:/Users/oluwa/Downloads/combined_words2.csv")
result.coalesce(1).write.csv("./combined_words3.csv")

In [ ]:
spark-submit "C:/Users/oluwa/Downloads/spark_run01.py"

spark-submit C:\Users\oluwa\Documents\hello_pyth\de-lab\spark_run01_local.py

In [ ]:
result.show(5)